In [233]:
import csv
import pandas as pd
import numpy as np
import sklearn

from pymongo import MongoClient

ALL_COLORS = ['W', 'U', 'B', 'R', 'G']

PATH = './data/draft_data_public.MKM.PremierDraft.csv'

NO_NO = 0
NO_YES = 1
YES_YES = 2

NO = 0
YES = 1

MAX_PIP_COUNT = 40
NUM_COLORS = 5
NUM_IN_PACK = 13
NUM_PACKS = 3

In [232]:
client = MongoClient()
cards_en = client.scryfall.cards_en

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    COLUMNS = next(draft_data)

NAMES = [col.split('pack_card_')[1] for col in columns if 'pack_card_' in col]
PICK_IDX = columns.index('pick')
DRAFTID_IDX = columns.index('draft_id')
PACKNUM_IDX = columns.index('pack_number')
PICKNUM_IDX = columns.index('pick_number')
PACK_IDX = columns.index('pack_card_A Killer Among Us')
POOL_IDX = columns.index('pool_A Killer Among Us')
NUM_CARDS = len(names)


In [205]:
def get_card(card_name):
    card = cards_en.find_one({'name': card_name})

    if card is None:
        query = {            
            'card_faces': {
                '$elemMatch': {
                    'name': card_name
                }
            },
        }

        card = cards_en.find_one(query)

    if card is None:
        error_message = 'No match for card name {}'.format(card_name)
        print(error_message)

    return(card)

def card_vector(index, row):
    return np.array([int(x) for x in row[index:index + NUM_CARDS]])

In [209]:
alsa_filepath = './data/alsa.npy'
# calculate ALSA

# last_seen = np.zeros(len(names))
# counts = np.zeros(len(names))
# with open(PATH) as csvfile:
#     draft_data = csv.reader(csvfile)
#     next(draft_data)
#     draft_id = None
#     pack_num = None
#     row_last_seen = np.zeros(len(names))
    
#     for row in draft_data:
#         if draft_id != row[DRAFTID_IDX] or pack_num != row[PACKNUM_IDX]:
#             last_seen += row_last_seen
#             counts += np.where(row_last_seen, 1, 0)
#             draft_id = row[DRAFTID_IDX]
#             pack_num = int(row[PACKNUM_IDX])
#             row_last_seen = np.zeros(NUM_CARDS)
#         pack = card_vector(PACK_IDX, row)
#         np.putmask(row_last_seen, pack, int(row[PICKNUM_IDX]))
        
# alsa = last_seen / counts
# alsa_series = pd.Series(alsa, index=NAMES)
# alsa_series = alsa_series[pd.notna(alsa)]

# with open(alsa_filepath, 'wb') as f:
#     np.save(f, alsa)

alsa = np.load(alsa_filepath)

In [192]:
len(names)

326

In [224]:
def pip_vector(card_name):
    card = get_card(card_name)
    if card is None:
        print(card_name + ' Not Found')
    mana_cost = card['mana_cost']
    return np.array([mana_cost.count(f'{{{color}}}') for color in ALL_COLORS])  \
        + 1/2 * np.array([mana_cost.count(f'/{color}') for color in ALL_COLORS])\
        + 1/2 * np.array([mana_cost.count(f'{color}/') for color in ALL_COLORS])

PIP_LOOKUP = np.array([pip_vector(name) for name in names])

def pool_pips(pool):
    return np.matmul(pool.reshape(1,NUM_CARDS), PIP_LOOKUP)[0]

def pick_number(row):
    return int(row[PICKNUM_IDX]) + NUM_IN_PACK * int(row[PACKNUM_IDX])

def pip_counts_to_matrix(pip_counts):
    matrix = np.zeros((NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
    for i in range(NUM_COLORS):
        matrix[i][0:int(pip_counts[i])] = 1
    return matrix
    
    


In [184]:
experiment_counts = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO, YES], NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
event_counts = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    for i in range(24):
        next(draft_data)
    
    for row in draft_data:
        pack = card_vector(PACK_IDX, row) 
        pack_alsa = np.where(pack, alsa, NUM_IN_PACK+1)
        pool = card_vector(POOOL_IDX, row)
        pool_pip_counts_pre = pool_pips(pool)
        pool_pre_matrix = pip_counts_to_matrix(pool_pip_counts_pre)
        pool_pip_counts_post = pool_pip_counts_pre + PIP_LOOKUP[names.index(row[PICK_IDX])]
        
        pool_post_matrix = pip_counts_to_matrix(pool_pip_counts_post)
        no_yes_matrix = pool_post_matrix - pool_pre_matrix
        no_no_matrix = 1 - pool_post_matrix
        
        min_indices = np.argpartition(pack_alsa, 2)
        min_alsa_card = min_indices[0]
        experiment_counts[pick_number(row)][min_alsa_card][YES] += pool_pre_matrix
        experiment_counts[pick_number(row)][min_alsa_card][NO] += 1 - pool_pre_matrix
        
        if (names[min_alsa_card] == row[columns.index('pick')])
            pool_after = pool_state_matrix + pip_matrix[min_alsa_card]
        

KeyboardInterrupt: 